# Rewrite-Retrieve-Read

You can also run this notebook online [on Noteable.io](https://app.noteable.io/published/d2cb020b-8341-4cff-b898-be33d3c62e21).

**Rewrite-Retrieve-Read** is a method proposed in the paper [Query Rewriting for Retrieval-Augmented Large Language Models](https://arxiv.org/pdf/2305.14283.pdf)

> Because the original query can not be always optimal to retrieve for the LLM, especially in the real world... we first prompt an LLM to rewrite the queries, then conduct retrieval-augmented reading

We show how you can easily do that with LangChain Expression Language in this notebook.

## Baseline

Baseline RAG (**Retrieve-and-read**) can be done like the following:

In [1]:
// Deno.env.set("OPENAI_API_KEY", "");
// Deno.env.set("TAVILY_API_KEY", "");

import { PromptTemplate } from "npm:langchain@0.0.172/prompts";
import { ChatOpenAI } from "npm:langchain@0.0.172/chat_models/openai";
import { StringOutputParser } from "npm:langchain@0.0.172/schema/output_parser";
import { RunnableSequence, RunnablePassthrough } from "npm:langchain@0.0.172/schema/runnable";
import { TavilySearchAPIRetriever } from "npm:langchain@0.0.172/retrievers/tavily_search_api";
import type { Document } from "npm:langchain@0.0.172/schema/document";

In [2]:
const template = `Answer the users question based only on the following context:

<context>
  {context}
</context>

Question: {question}`

const prompt = PromptTemplate.fromTemplate(template);

const model = new ChatOpenAI({
  model: "o4-mini",
  temperature: 0,
  apiKey: Deno.env.get("OPENAI_API_KEY"),
})

const retriever = new TavilySearchAPIRetriever({
  k: 3,
  apiKey: Deno.env.get("TAVILY_API_KEY"),
});

const formatDocs = (documents: Document[]) => documents.map((doc) => doc.pageContent).join("\n");

In [3]:
const chain = RunnableSequence.from([
  {
    context: retriever.pipe(formatDocs),
    question: new RunnablePassthrough()
  },
  prompt,
  model,
  new StringOutputParser()
]);

In [4]:
const simpleQuery = "what is langchain?";

await chain.invoke(simpleQuery);

"LangChain is an open source framework designed to simplify the creation of applications using large "... 217 more characters

While this is fine for well formatted queries, it can break down for more complicated queries

In [5]:
const distractedQuery = "man that sam bankman fried trial was crazy! what is langchain?";

await chain.invoke(distractedQuery);

'Based on the given context, there is no information about "langchain" or any relevance to Sam Bankma'... 16 more characters

This is because the retriever does a bad job with these "distracted" queries:

In [6]:
await retriever.invoke(distractedQuery);

[
  Document {
    pageContent: "Three of Sam Bankman-Fried's closest former associates have turned against him, and their testimony "... 98 more characters,
    metadata: {
      title: "3 key witnesses who could send Sam Bankman-Fried to prison for life : NPR",
      source: "https://www.npr.org/2023/10/21/1207143248/sam-bankman-fried-trial-ftx-crypto-fraud-alameda",
      score: 0.95083,
      images: null
    }
  },
  Document {
    pageContent: "October 18, 20232:46 PM PDTUpdated 2 days ago Former FTX Chief Executive Sam Bankman-Fried, who face"... 94 more characters,
    metadata: {
      title: "Sam Bankman-Fried trial jury sees his profane messages about regulators ...",
      source: "https://www.reuters.com/legal/sam-bankman-fried-trial-jury-sees-his-profane-message-about-regulators"... 12 more characters,
      score: 0.91945,
      images: null
    }
  },
  Document {
    pageContent: "FTX founder Sam Bankman-Fried leaves the courthouse following his arraignment in New York

## Rewrite-Retrieve-Read Implementation

The main part is a rewriter to rewrite the search query:

In [7]:
const rewriteTemplate = `Provide a better search query for \
web search engine to answer the given question, end \
the queries with ’**’. Question: \
{x} Answer:`;
const rewritePrompt = PromptTemplate.fromTemplate(rewriteTemplate);

In [8]:
// Parser to remove the `**`
const _parse = (text) => text.replace("**", "");

In [9]:
// rewriter = rewrite_prompt | ChatOpenAI(temperature=0) | StrOutputParser() | _parse
const rewriter = RunnableSequence.from([
  rewritePrompt,
  new ChatOpenAI({ model: "o4-mini", temperature: 0 }),
  new StringOutputParser(),
  _parse
]);

In [10]:
await rewriter.invoke({"x": distractedQuery})

"What is the definition and purpose of Langchain?"

In [11]:
const rewriteRetrieveReadChain = RunnableSequence.from([
  {
    context: RunnableSequence.from([
      { x: new RunnablePassthrough() },
      rewriter,
      retriever,
      formatDocs,
    ]),
    question: new RunnablePassthrough()
  },
  prompt,
  model,
  new StringOutputParser()
]);

In [12]:
await rewriteRetrieveReadChain.invoke(distractedQuery);

"LangChain is a framework designed to simplify the creation of applications using large language mode"... 293 more characters